In [50]:
from math import sqrt
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from pydataset import data
import statistics
import acquire
import prepare

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

import explore
import warnings
warnings.filterwarnings("ignore")

import graphviz
from graphviz import Graph

### Using the titanic data, in your classification-exercises repository, create a notebook, model.ipynb where you will do the following:

In [2]:
train, validate, test = prepare.prep_titanic_data(acquire.get_titanic_data())

In [3]:
train.head(3)

,survived,pclass,sibsp,parch,fare,alone,sex_male,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
455,1,3,0,0,7.8958,1,1,1,0,0
380,1,1,0,0,227.5250,1,0,1,0,0
492,0,1,0,0,30.5000,1,1,0,0,1


### What is your baseline prediction?
### What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode).

In [4]:
#train.survived.value_counts()

In [5]:
#train["baseline"] = 0

In [6]:
#(train.survived==train.baseline).mean()

Baseline Model is .616

### When you make those predictions, what is your accuracy? This is your baseline accuracy.
    61% and yes this is my basline accuracy assuming no survivers 

### Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534 entries, 455 to 496
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   survived                 534 non-null    int64  
 1   pclass                   534 non-null    int64  
 2   sibsp                    534 non-null    int64  
 3   parch                    534 non-null    int64  
 4   fare                     534 non-null    float64
 5   alone                    534 non-null    int64  
 6   sex_male                 534 non-null    uint8  
 7   embark_town_Cherbourg    534 non-null    uint8  
 8   embark_town_Queenstown   534 non-null    uint8  
 9   embark_town_Southampton  534 non-null    uint8  
dtypes: float64(1), int64(5), uint8(4)
memory usage: 31.3 KB


In [8]:
# create X & y version of train, where y is a series with just the target variable and X are all the features. 

X_train = train.drop(columns=['survived'])
y_train = train.survived

X_validate = validate.drop(columns=['survived'])
y_validate = validate.survived

X_test = test.drop(columns=['survived'])
y_test = test.survived

In [9]:
clf = DecisionTreeClassifier(max_depth=3, random_state=123)

In [10]:
len(X_train.columns)

9

In [11]:
clf = clf.fit(X_train, y_train)

In [12]:
y_pred = clf.predict(X_train)
y_pred[0:5]

array([0, 1, 0, 0, 0])

In [13]:
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba[0:5]

array([[0.905     , 0.095     ],
       [0.01785714, 0.98214286],
       [0.69105691, 0.30894309],
       [0.69105691, 0.30894309],
       [0.905     , 0.095     ]])

### Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [14]:
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
      .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 0.82


In [15]:
confusion_matrix(y_train, y_pred)

array([[295,  34],
       [ 63, 142]])

In [16]:
y_train.value_counts()

0    329
1    205
Name: survived, dtype: int64

In [17]:
labels = sorted(y_train.unique())

pd.DataFrame(confusion_matrix(y_train, y_pred), index=labels, columns=labels)

,0,1
0,295,34
1,63,142


In [18]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.90      0.86       329
           1       0.81      0.69      0.75       205

    accuracy                           0.82       534
   macro avg       0.82      0.79      0.80       534
weighted avg       0.82      0.82      0.82       534



In [19]:
dot_data = export_graphviz(clf, feature_names= X_train.columns,class_names=['died','lived'], rounded=True, filled=True, out_file=None)
graph = graphviz.Source(dot_data) 

graph.render('titanic_decision_tree', view=True)

'titanic_decision_tree.pdf'

# Run through steps 2-4 using a different max_depth value.

In [20]:
def train_model(max_depth):
    clf = DecisionTreeClassifier(max_depth=max_depth, random_state=123)
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_train)
    y_pred_proba = clf.predict_proba(X_train)
    accuracy = clf.score(X_train, y_train)
    #print(classification_report(y_train, y_pred))
    return accuracy

def validate_model(max_depth):
    clf = DecisionTreeClassifier(max_depth=max_depth, random_state=123)
    clf = clf.fit(X_train, y_train)
    y_pred = clf.predict(X_validate)
    y_pred_proba = clf.predict_proba(X_validate)
    accuracy = clf.score(X_validate, y_validate)
    return accuracy
    #print(classification_report(y_validate, y_pred))

In [21]:
# for depth in range(1,7):
#     print(f'Accuracy of Decision Tree classifier in depth: {depth}: train = {round(train_model(depth), 3)} validate = {round(validate_model(depth), 3)}')

### Which model performs better on your in-sample data?
    Max Depth 6 worked better in my sample data

### Which model performs best on your out-of-sample data, the validate set?
    Max Depth 2 was the closest accuracy to my train set

# Random Forest Exercises

### Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.

In [22]:
rf = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=1,
                            n_estimators=100,
                            max_depth=10, 
                            random_state=123)

In [23]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=123)

### Evaluate your results using the model score, confusion matrix, and classification report.

In [24]:
y_pred = rf.predict(X_train)
y_pred[0:5]

array([0, 1, 0, 1, 0])

In [25]:
y_pred_proba = rf.predict_proba(X_train)
y_pred_proba

array([[0.78429739, 0.21570261],
       [0.        , 1.        ],
       [0.51708909, 0.48291091],
       ...,
       [0.87725658, 0.12274342],
       [0.94299092, 0.05700908],
       [0.00230769, 0.99769231]])

In [26]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf.score(X_train, y_train)))

Accuracy of random forest classifier on training set: 0.94


In [27]:
print(confusion_matrix(y_train, y_pred))

[[324   5]
 [ 25 180]]


In [28]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.98      0.96       329
           1       0.97      0.88      0.92       205

    accuracy                           0.94       534
   macro avg       0.95      0.93      0.94       534
weighted avg       0.95      0.94      0.94       534



In [29]:
print('Accuracy of random forest classifier on test set: {:.2f}'
     .format(rf.score(X_validate, y_validate)))

Accuracy of random forest classifier on test set: 0.76


In [30]:
train.columns

Index(['survived', 'pclass', 'sibsp', 'parch', 'fare', 'alone', 'sex_male',
       'embark_town_Cherbourg', 'embark_town_Queenstown',
       'embark_town_Southampton'],
      dtype='object')

In [31]:
rf.feature_importances_

array([0.1043249 , 0.06451271, 0.04902207, 0.38835831, 0.02167164,
       0.31666719, 0.01491566, 0.014347  , 0.02618052])

1. fare
2. sex_male
3. pclass

In [32]:
def forest_train(leaf, depth):
    rf = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=leaf,
                            n_estimators=100,
                            max_depth=depth, 
                            random_state=123)
    rf.fit(X_train, y_train)
    accuracy = rf.score(X_train, y_train)
    return round(accuracy, 3)

def forest_validate(leaf, depth):
    rf = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=leaf,
                            n_estimators=100,
                            max_depth=depth, 
                            random_state=123)
    rf.fit(X_train, y_train)
    accuracy = rf.score(X_validate, y_validate)
    return round(accuracy, 3)

In [33]:
# for leaf in range(1,11):
#     for depth in range(1,11):
#         print(f"For leaf: {leaf} and depth: {depth} accuracy is: {forest_train(leaf, depth)}")
#         print(f"For leaf: {leaf} and depth: {depth} accuracy is: {forest_validate(leaf, depth)}")
#         print(f"Difference is: {forest_train(leaf, depth) - forest_validate(leaf, depth)}")

# KNN

### Fit a K-Nearest Neighbors classifier to your training sample and transform (i.e. make predictions on the training sample)

In [34]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

In [35]:
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [36]:
y_pred = knn.predict(X_train)

In [37]:
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))

Accuracy of KNN classifier on training set: 0.82


### Evaluate your results using the model score, confusion matrix, and classification report.

    Model Score

In [38]:
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))

Accuracy of KNN classifier on training set: 0.82


    Confusion Matrix

In [39]:
print(confusion_matrix(y_train, y_pred))

[[287  42]
 [ 55 150]]


    Classification Report

In [40]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.87      0.86       329
           1       0.78      0.73      0.76       205

    accuracy                           0.82       534
   macro avg       0.81      0.80      0.81       534
weighted avg       0.82      0.82      0.82       534



### Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [42]:
report = classification_report(y_train, y_pred, output_dict=True)
print(pd.DataFrame(report))
print()

                    0           1  accuracy   macro avg  weighted avg
precision    0.839181    0.781250  0.818352    0.810216      0.816942
recall       0.872340    0.731707  0.818352    0.802024      0.818352
f1-score     0.855440    0.755668  0.818352    0.805554      0.817138
support    329.000000  205.000000  0.818352  534.000000    534.000000



### Run through steps 2-4 setting k to 10

In [43]:
def knn(k):
    knn = KNeighborsClassifier(n_neighbors=k, weights='uniform')
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_train)
    report = classification_report(y_train, y_pred, output_dict=True)
    print(f"K as {k}")
    print(pd.DataFrame(report))

In [46]:
knn(10)

K as 10
                    0           1  accuracy   macro avg  weighted avg
precision    0.783198    0.757576  0.775281    0.770387      0.773362
recall       0.878419    0.609756  0.775281    0.744088      0.775281
f1-score     0.828080    0.675676  0.775281    0.751878      0.769573
support    329.000000  205.000000  0.775281  534.000000    534.000000


### Run through setps 2-4 setting k to 20

In [45]:
knn(20)

K as 20
                    0           1  accuracy   macro avg  weighted avg
precision    0.751309    0.723684  0.743446    0.737497      0.740704
recall       0.872340    0.536585  0.743446    0.704463      0.743446
f1-score     0.807314    0.616246  0.743446    0.711780      0.733964
support    329.000000  205.000000  0.743446  534.000000    534.000000


### What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

    Accuracy is going down as you have more neighbors in the in-sample data. Probably because the calculated distance is not
    as seperated as one might think.

### Which model performs best on our out-of-sample data from validate?

---

# <b>Logistic Regression</b>

### For all of the models you create, choose a threshold that optimizes for accuracy.

### Do your work for these exercises in either a notebook or a python script named model within your classification-exercises repository. Add, commit, and push your work.

### Create a model that includes age in addition to fare and pclass. Does this model perform better than your baseline?

In [48]:
train, validate, test = prepare.prep_titanic_data(acquire.get_titanic_data())

In [49]:
X_train = train.drop(columns=['survived'])
y_train = train.survived

X_validate = validate.drop(columns=['survived'])
y_validate = validate.survived

X_test = test.drop(columns=['survived'])
y_test = test.survived

In [52]:
logit = LogisticRegression(C=1, class_weight={0:1, 1:99}, random_state=123, intercept_scaling=1, solver='lbfgs')
logit.fit(X_train, y_train)

LogisticRegression(C=1, class_weight={0: 1, 1: 99}, random_state=123)

In [53]:
y_pred = logit.predict(X_train)

In [54]:
y_pred_proba = logit.predict_proba(X_train)

In [55]:
print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(logit.score(X_train, y_train)))

Accuracy of Logistic Regression classifier on training set: 0.39


### Include sex in your model as well. Note that you'll need to encode or create a dummy variable of this feature before including it in a model.

### Try out other combinations of features and models.

### Use you best 3 models to predict and evaluate on your validate sample.

### Choose you best model from the validation performation, and evaluate it on the test dataset. How do the performance metrics compare to validate? to train?